In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

In [3]:
config = '/home/ubuntu/malay_vits/haqkiem.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.filter_length // 2 + 1,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [7]:
_ = model.eval()

In [8]:
!ls /home/ubuntu/malay_vits/logs/haqkiem/G_*.pth

/home/ubuntu/malay_vits/logs/haqkiem/G_0.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_100000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_10000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_1000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_101000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_102000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_103000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_104000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_105000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_106000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_107000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_108000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_109000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_110000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_11000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_111000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_112000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_113000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_114000.pth
/home/ubuntu/malay_vits/logs/haqkiem/G_115000.pth
/home/ubu

In [9]:
pth = '/home/ubuntu/malay_vits/logs/haqkiem/G_122000.pth'

In [10]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [11]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [12]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None)

In [13]:
text = 'Haqkiem adalah pelajar tahun akhir yang mengambil Ijazah Sarjana Muda Sains Komputer Kecerdasan Buatan utama dari Universiti Teknikal Malaysia Melaka (UTeM) yang kini berusaha untuk latihan industri di mana dia secara praktikal dapat menerapkan pengetahuannya dalam Perisikan Perisian dan Pengaturcaraan ke arah organisasi atau industri yang berkaitan.'

In [14]:
text = """
SHAH ALAM - Pertubuhan Kebajikan Anak Bersatu Selangor (PKABS) bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja (RCI) untuk menyiasat isu kartel daging.

Pengerusinya, Rahmadin Alimuddin berkata, perkara tersebut perlu disiasat sebaiknya kerana isu logo halal palsu membimbangkan umat Islam di negara ini.

Menurutnya, siasatan juga dapat memastikan pembekal daging beku mengikut piawaian yang ditetapkan oleh kerajaan Malaysia sebelum menjualnya di pasaran.
"""

In [15]:
t, ids = normalizer.normalize(text)
t

'shah alam - pertubuhan kebajikan anak bersatu selangor , pkabs , bersetuju pihak kerajaan mewujudkan suruhanjaya siasatan diraja , rci , untuk menyiasat isu kartel daging . pengerusinya , rahmadin alimuddin berkata , perkara tersebut perlu disiasat sebaiknya kerana isu logo halal palsu membimbangkan umat islam di negara ini . menurutnya , siasatan juga dapat memastikan pembekal daging beku mengikut piawaian yang ditetapkan oleh kerajaan malaysia sebelum menjualnya di pasaran .'

In [16]:
ids = intersperse(ids, 0)

In [17]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [18]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.6666,
    noise_scale_w=0.66666,
    length_scale=1.0,
)

In [19]:
audio = audio[0][0,0].detach().numpy()

In [20]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [21]:
torch.save(model.state_dict(), 'haqkiem.pth')

In [22]:
from malaya_boilerplate.huggingface import upload_dict

In [ ]:
files_mapping = {'haqkiem.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-haqkiem', files_mapping = files_mapping, username = 'mesolitica')

/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.7. Pass `repo_id` instead.
  warnings.warn(
409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this model repo
